# CM4 velocity magnitude across Solomon Strait 
*Akira Di Sandro, 6/15/20*
<br>
In this notebook, I will be attempting to calculate the velocity magnitude across the cross-section of the Solomon Sea.

## 1. Import Python Packages

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [17]:
import xarray as xr
xr.set_options(display_style='html')
import intake
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np

## 2. Import CM4 data from CMIP6 dataset

In [9]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)

In [10]:
dog = col.search(source_id='GFDL-CM4', experiment_id='historical', table_id='Omon', variable_id=['uo','vo','wo'], grid_label='gn')

In [11]:
dset_dict = dog.to_dataset_dict(zarr_kwargs={'consolidated': True})
list(dset_dict.keys())

Dataset(s):   0%|                                        | 0/1 [00:00<?, ?it/s]


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


Dataset(s): 100%|████████████████████████████████| 1/1 [00:00<00:00,  1.81it/s]


['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn']

In [12]:
CM4_hist = dset_dict['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn']
CM4_hist

<xarray.Dataset>
Dimensions:    (bnds: 2, lev: 35, member_id: 1, time: 1980, x: 1440, y: 1080)
Coordinates:
    time_bnds  (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(35, 2), meta=np.ndarray>
    lon        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lat        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * lev        (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * bnds       (bnds) float64 1.0 2.0
  * member_id  (member_id) <U8 'r1i1p1f1'
Data variables:
    uo         (member_id, time, lev, y, x) float32 dask.array<chunksize=(1, 1, 35, 1080, 1440), meta=np.ndarray>
    vo         (member_id, time, lev, y, x) float32 dask.array<chunksize=(1, 1, 35, 1080, 1440), meta=np.ndarray>
Attributes:
    grid:                    ocean data on native grid tripolar - nominal 0.2...
    realm:                   ocean
    source_id:               GFDL-CM4
    sub_experiment:          none
    initialization_index:    1
    references:              see further_info_url attribute
    grid_label:              gn
    branch_method:           standard
    comment:                 <null ref>
    institution:             National Oceanic and Atmospheric Administration,...
    parent_activity_id:      CMIP
    license:                 CMIP6 model data produced by NOAA-GFDL is licens...
    branch_time_in_child:    0.0
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.G...
    experiment:              historical
    data_specs_version:      01.00.27
    variant_info:            N/A
    title:                   NOAA GFDL GFDL-CM4 model output prepared for CMI...
    activity_id:             CMIP
    physics_index:           1
    mip_era:                 CMIP6
    branch_time_in_parent:   36500.0
    parent_source_id:        GFDL-CM4
    table_id:                Omon
    frequency:               mon
    status:                  2019-08-06;created;by nhn2@columbia.edu
    product:                 model-output
    parent_experiment_id:    piControl
    parent_mip_era:          CMIP6
    contact:                 gfdl.climate.model.info@noaa.gov
    Conventions:             CF-1.7 CMIP-6.0 UGRID-1.0
    institution_id:          NOAA-GFDL
    tracking_id:             hdl:21.14100/d0ebcd36-d2cd-442e-a8d9-8f7a27168a2...
    experiment_id:           historical
    parent_time_units:       days since 0001-1-1
    forcing_index:           1
    source_type:             AOGCM
    history:                 File was processed by fremetar (GFDL analog of C...
    variant_label:           r1i1p1f1
    source:                  GFDL-CM4 (2018): \naerosol: interactive\natmos: ...
    sub_experiment_id:       none
    parent_variant_label:    r1i1p1f1
    nominal_resolution:      25 km
    realization_index:       1
    intake_esm_dataset_key:  CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn

* Sidenote: I just realized that taking the magnitude of the velocity components at this cross section is a little meaningless because the cross-section is along a line of latitude, meaning the 'uo' or u-component (zonal) velocity will add nothing to the magnitude (sin(0)=0) while the 'vo' or v-component (meridional) velocity will be the magnitude itself (sine(pi/2)=1).

* This means I need to first figure out a way to make a diagonal cross-section of the Solomon Sea (or maybe directly of the Solomon Strati).

## 3. Make Function to find Magnitude at every (x,y)

In [ ]:
mag_vel = np.empty([len(x),len(y)])   #create empty matrix where we'll store mag for every (x,y)
   #making the function to calculate magnitude
def calcmag(arr, x, y, uo, vo):   
    for i in range(len(x):
        for j in range(len(y):
            